# Textgenerierung

Auch in dieser Woche beschäftigen wir uns mit Twitter-Daten. Diesmal geht es jedoch nicht um Sentiment Analysis, sondern wir wollen ein Modell trainieren, das es uns erlaubt, Tweets in einem bestimmten Stil zu generieren.

Als Datengrundlage dienen uns Daten von http://www.trumptwitterarchive.com. Brendan Brown, der Betreiber der Seite hat sämtliche Tweets von Donald Trump seit Mai 2009 zusammengetragen. Da wir die Sprache des US-Präsidenten modellieren wollen, verwenden wir nur dessen eigene und keine Retweets.

Unser Modell wird auf der Ebene von Einzelzeichen arbeiten, zunächst wollen wir uns aber auf einer höheren Ebene einen Überblick über den Datensatz verschaffen.

## 1. Aufgabe: Überblick über den Datensatz
### 1.1 Datensatz einlesen
Lest den in der Datei ```all_tweets.json``` enthaltenen Datensatz in einen Pandas-Dataframe mit folgenden Spalten ein: ```created_at```, ```id```, ```text```. Die übrigen im JSON enthaltenen Felder können ignoriert werden.



In [0]:
import pandas as pd
import json
from google.colab import drive

drive.mount('/data')

tweets = pd.read_json('/data/My Drive/Colab Notebooks/data/all_tweets.json')

tweets

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /data


,source,text,created_at,retweet_count,favorite_count,is_retweet,id_str
0,Twitter for iPhone,Will be interviewed by @MariaBartiromo on @Fox...,2019-04-28 12:59:53+00:00,4392,15808,0.0,1122485588580605952
1,Twitter for iPhone,"....for the more traditional, but not very bri...",2019-04-28 03:10:25+00:00,13425,60142,0.0,1122337243744497664
2,Twitter for iPhone,"The Democratic National Committee, sometimes r...",2019-04-28 03:10:24+00:00,13243,57238,0.0,1122337240330297344
3,Twitter for iPhone,....Ever since Andrew came to my office to ask...,2019-04-28 02:57:32+00:00,15124,58221,0.0,1122334000519868416
4,Twitter for iPhone,Thank you to brilliant and highly respected at...,2019-04-28 02:57:31+00:00,15886,65548,0.0,1122333996451418112
...,...,...,...,...,...,...,...
25690,Twitter Web Client,"""My persona will never be that of a wallflower...",2009-05-12 14:07:28+00:00,1421,1950,0.0,1773561338
25691,Twitter Web Client,New Blog Post: Celebrity Apprentice Finale and...,2009-05-08 20:40:15+00:00,8,27,0.0,1741160716
25692,Twitter Web Client,Donald Trump reads Top Ten Financial Tips on L...,2009-05-08 13:38:08+00:00,3,2,0.0,1737479987
25693,Twitter Web Client,Donald Trump will be appearing on The View tom...,2009-05-05 01:00:10+00:00,2,3,0.0,1701461182


### 1.2 Jahr hinzufügen
Für unsere Auswertungen interessiert uns nur das Jahr, in dem der Tweet verfasst wurde, nicht das genaue Datum. Wir fügen daher dem Dataframe eine zusätzliche Spalte ```year``` hinzu. Hierbei kann zum Beispiel die Pandas-Funktion ```DatetimeIndex``` verwendet werden, um aus dem ```created_at``` String einen DatetimeIndex zu machen, auf dessen einzelne Felder (```year```, ```month```, ```day```, ...) dann mittels Punktoperator zugegriffen werden kann.

In [0]:
tweets['year'] = tweets.apply(lambda row: pd.to_datetime(row['created_at']).year, axis=1)
tweets

,source,text,created_at,retweet_count,favorite_count,is_retweet,id_str,year
0,Twitter for iPhone,Will be interviewed by @MariaBartiromo on @Fox...,2019-04-28 12:59:53+00:00,4392,15808,0.0,1122485588580605952,2019
1,Twitter for iPhone,"....for the more traditional, but not very bri...",2019-04-28 03:10:25+00:00,13425,60142,0.0,1122337243744497664,2019
2,Twitter for iPhone,"The Democratic National Committee, sometimes r...",2019-04-28 03:10:24+00:00,13243,57238,0.0,1122337240330297344,2019
3,Twitter for iPhone,....Ever since Andrew came to my office to ask...,2019-04-28 02:57:32+00:00,15124,58221,0.0,1122334000519868416,2019
4,Twitter for iPhone,Thank you to brilliant and highly respected at...,2019-04-28 02:57:31+00:00,15886,65548,0.0,1122333996451418112,2019
...,...,...,...,...,...,...,...,...
25690,Twitter Web Client,"""My persona will never be that of a wallflower...",2009-05-12 14:07:28+00:00,1421,1950,0.0,1773561338,2009
25691,Twitter Web Client,New Blog Post: Celebrity Apprentice Finale and...,2009-05-08 20:40:15+00:00,8,27,0.0,1741160716,2009
25692,Twitter Web Client,Donald Trump reads Top Ten Financial Tips on L...,2009-05-08 13:38:08+00:00,3,2,0.0,1737479987,2009
25693,Twitter Web Client,Donald Trump will be appearing on The View tom...,2009-05-05 01:00:10+00:00,2,3,0.0,1701461182,2009


### 1.3 Textlänge analysieren
Naturgemäß gibt es bei Tweets nur eine begrenzte Varianz, was die Textlänge angeht. Wir wollen uns dennoch anschauen, wie sich die Textlänge im Laufe der Jahre entwickelt hat.
Dazu fügen wir unserem Dataframe zunächst eine Spalte ```text_length``` hinzu, in der wir festhalten, welche Länge der jeweilige Tweet-Text hat.

**Hinweis**
Mittels ```apply``` lassen sich Funktionen auf Spalten des Dataframes mappen: ```df['new'] = df['old'].apply(lambda x : fancy_stuff(x))```

In [0]:
tweets['text_length'] = tweets.apply(lambda row: len(row['text']), axis=1) # TODO Spalte füllen
tweets

,source,text,created_at,retweet_count,favorite_count,is_retweet,id_str,year,text_length
0,Twitter for iPhone,Will be interviewed by @MariaBartiromo on @Fox...,2019-04-28 12:59:53+00:00,4392,15808,0.0,1122485588580605952,2019,191
1,Twitter for iPhone,"....for the more traditional, but not very bri...",2019-04-28 03:10:25+00:00,13425,60142,0.0,1122337243744497664,2019,177
2,Twitter for iPhone,"The Democratic National Committee, sometimes r...",2019-04-28 03:10:24+00:00,13243,57238,0.0,1122337240330297344,2019,144
3,Twitter for iPhone,....Ever since Andrew came to my office to ask...,2019-04-28 02:57:32+00:00,15124,58221,0.0,1122334000519868416,2019,214
4,Twitter for iPhone,Thank you to brilliant and highly respected at...,2019-04-28 02:57:31+00:00,15886,65548,0.0,1122333996451418112,2019,145
...,...,...,...,...,...,...,...,...,...
25690,Twitter Web Client,"""My persona will never be that of a wallflower...",2009-05-12 14:07:28+00:00,1421,1950,0.0,1773561338,2009,109
25691,Twitter Web Client,New Blog Post: Celebrity Apprentice Finale and...,2009-05-08 20:40:15+00:00,8,27,0.0,1741160716,2009,103
25692,Twitter Web Client,Donald Trump reads Top Ten Financial Tips on L...,2009-05-08 13:38:08+00:00,3,2,0.0,1737479987,2009,116
25693,Twitter Web Client,Donald Trump will be appearing on The View tom...,2009-05-05 01:00:10+00:00,2,3,0.0,1701461182,2009,131


Für einen groben Überblick schauen wir uns einige Kennzahlen zur Textlänge an. Dazu gruppieren wir nach ```year```und nutzen dann die ```describe```-Methode des Dataframes, wobei wir nur Spalten vom Typ ```numpy.number``` betrachten und daher der ```describe```-Methode eine entsprechende ```include```-Liste mitgeben.

In [0]:
import numpy as np
tweets.groupby(tweets['year']).describe(include=[np.number])

retweet_count                              ... text_length               
             count          mean           std  ...         50%     75%    max
year                                            ...                           
2009          56.0     39.071429    191.165503  ...       115.0  126.25  140.0
2010         142.0     79.380282    417.097668  ...       132.5  138.00  140.0
2011         774.0    187.506460    556.265788  ...       114.0  133.00  140.0
2012        3530.0    457.685836   3625.044171  ...       117.0  135.00  148.0
2013        5775.0    239.694719   1613.828343  ...        94.0  128.00  152.0
2014        2433.0    332.652692   1283.491376  ...       124.0  136.00  148.0
2015        3057.0   1131.147203   1549.863696  ...       120.0  137.00  155.0
2016        3465.0   8374.749784  10837.525448  ...       127.0  138.00  148.0
2017        2284.0  19504.978984  14416.083801  ...       138.0  143.00  320.0
2018        3104.0  21092.619201  10443.373373  ...       221.0  277.00  315.0
2019        1075.0  24635.476279  10951.524073  ...       205.0  276.00  302.0

[11 rows x 48 columns]

### 1.4. Top-Hashtags und -Mentions
Nachdem wir uns mit der Länge der Texte beschäftigt haben, wollen wir nun herausfinden, wen Donald Trump in seinen Tweets erwähnt und welche Themen er (hash)taggt. Dabei interessiert uns die Entwicklung über die Jahre.

Anbei ein Vorschlag bezüglich des Vorgehens:
Wir beginnen mit den Hashtags und verwenden zunächst einen kleinen Trick, um ein Dictionary zu erstellen, das für jedes Jahr einen "Sub-Dataframe" enthält. 

Aus diesen Dataframes extrahieren wir dann pro Jahr alle Tweettexte in Form eines einzelnen Strings. Dazu konkatenieren wir die ```text```-Felder der Dataframes per ```' '.join(frame['text']) for frame in ...```

Damit haben wir ein Dictionary, das pro Jahr alle konkatenierten Tweet-Texte enthält, aus denen wir dann die Hashtags extrahieren können. 
Hierbei machen wir uns noch keine allzu großen Gedanken über Normalisierung, sondern zerlegen die langen Texte einfach per ```split()``` in einzelne Tokens, aus denen wir dann die Hashtags herausfiltern. 
Um die Top-Hashtags in Erfahrung zu bringen, verwenden wir wieder ```Counter```.

In [0]:
from collections import Counter
import re

tweets_by_year = dict(list(tweets.groupby(['year'])))
texts_per_year = {}

for item in tweets_by_year.items():
  texts_per_year.update({item[0]: ' '.join(item[1]['text'])})

top_hashtags_per_year= {}
for item in texts_per_year.items():
  #tokens = re.split(' |, |\.*|\\n|! *|: ', item[1].encode('ascii', 'ignore').decode('ascii'))# item[1].split(' ')
  replaced_string = re.sub(r'\.|!|:|\n|-|,|\?|\'', ' ', item[1].encode('ascii', 'ignore').decode('ascii'))
  replaced_string = re.sub(r'#', ' #', replaced_string)
  tokens = replaced_string.split(' ')
  hashtags = [token for token in tokens if '#' in token]
  top_hashtags_per_year.update({item[0]: Counter(hashtags)})

for item in top_hashtags_per_year.items():
  print(item[0], item[1].most_common(10))

2009 []
2010 [('#EvanForSI', 1)]
2011 [('#TimeToGetTough', 59), ('#trumpvlog', 43), ('#1', 7), ('#TrumpRoast', 4), ('#Apprentice', 3), ('#TImeToGetTough', 2), ('#badratings', 2), ('#MissUSA', 2), ('#VEGASusa11', 2), ('#noratings', 1)]
2012 [('#TimeToGetTough', 33), ('#trumpvlog', 19), ('#sweepstweet', 19), ('#CelebrityApprentice', 14), ('#TrumpTuesday', 13), ('#CelebApprentice', 13), ('#1', 11), ('#MissUniverse', 9), ('#TRUMP', 8), ('#VPDebate', 8)]
2013 [('#CelebApprentice', 128), ('#1', 29), ('#WWEHOF', 18), ('#TrumpVine', 9), ('#TBT', 7), ('#MakeDCListen', 6), ('#FundAnything', 6), ('#CPAC2013', 4), ('#2', 3), ('#RoadHard', 3)]
2014 [('#Oscars', 19), ('#TBT', 13), ('#1', 13), ('#TrumpVlog', 12), ('#FreeOurMarine', 6), ('#FlashbackFriday', 5), ('#TrumpAdvice', 5), ('#BlueMonster', 4), ('#IceBucketChallenge', 4), ('#BringBackOurMarine', 4)]
2015 [('#MakeAmericaGreatAgain', 127), ('#Trump2016', 115), ('#1', 23), ('#CelebApprentice', 20), ('#GOPDebate', 12), ('#FITN', 10), ('#asktrump',

Als nächstes interessieren uns die Mentions. Wir können hier analog zu den Hashtags vorgehen.

In [0]:
top_mentions_per_year = {} # TODO

for item in texts_per_year.items():
  replaced_string = re.sub(r'\.|!|:|\n|-|,|\?|\'', ' ', item[1].encode('ascii', 'ignore').decode('ascii'))
  replaced_string = re.sub(r'@', ' @', replaced_string)
  tokens = replaced_string.split(' ')
  hashtags = [token for token in tokens if '@' in token and len(token) > 1]
  top_mentions_per_year.update({item[0]: Counter(hashtags)})

for item in top_mentions_per_year.items():
  print(item[0], item[1].most_common(10))

2009 [('@IvankaTrump', 1)]
2010 [('@kingsthings', 1), ('@bretmichaels', 1), ('@hollyrpeete', 1)]
2011 [('@BarackObama', 155), ('@FoxNews', 18), ('@iontv', 17), ('@Israel', 14), ('@gretawire', 12), ('@foxandfriends', 8), ('@Lawrence', 7), ('@johnboehner', 6), ('@JonHuntsman', 5), ('@TeamCavuto', 5)]
2012 [('@BarackObama', 544), ('@MittRomney', 288), ('@CelebApprentice', 45), ('@SquawkCNBC', 40), ('@foxandfriends', 38), ('@gretawire', 26), ('@Yankees', 25), ('@RickSantorum', 22), ('@Lord_Sugar', 21), ('@Rosie', 20)]
2013 [('@ApprenticeNBC', 106), ('@realDonaldTrump', 91), ('@billmaher', 66), ('@CelebApprentice', 62), ('@GOP', 36), ('@AlexSalmond', 34), ('@IvankaTrump', 33), ('@TrumpTowerNY', 31), ('@THEGaryBusey', 30), ('@DannyZuker', 29)]
2014 [('@TrumpDoral', 36), ('@AGSchneiderman', 30), ('@foxandfriends', 29), ('@TrumpChicago', 22), ('@TrumpNewYork', 19), ('@ApprenticeNBC', 18), ('@TrumpTurnberry', 16), ('@AlexSalmond', 15), ('@Macys', 13), ('@Newsmax_Media', 13)]
2015 [('@FoxNews', 

### 1.5 Vokabular
Bevor wir endgültig auf die Ebene der Einzelzeichen herabsteigen, wollen wir uns das von Trump verwendete Vokabular genauer ansehen und dabei auch herausfinden, ob weitere Vorverarbeitungsschritte nötig sind.

Dazu erstellen wir uns zunächst eine Liste aller Tokens, die in den Dokumenten vorkommen. Wir gruppieren nicht mehr per Jahr, sondern gehen ganz simpel vor und konkatenieren alle Tweet-Texte in einen langen String, den wir dann in einzelne Terme splitten.

Gebt die 20 häufigsten Terme aus. Was fällt auf (insbesondere bei Betrachtung des hinteren Endes der Liste)?

In [0]:
tweet_texts = ' '.join(tweets['text'])
tokens = tweet_texts.split()

Counter(tokens).most_common(20)
# TODO Top-20 Terme

[('the', 17252),
 ('to', 11653),
 ('and', 9303),
 ('of', 8246),
 ('a', 7975),
 ('is', 7535),
 ('in', 6777),
 ('for', 5067),
 ('I', 4861),
 ('on', 4842),
 ('be', 4023),
 ('will', 3761),
 ('that', 3217),
 ('are', 3177),
 ('with', 2980),
 ('you', 2963),
 ('at', 2800),
 ('&amp;', 2764),
 ('The', 2759),
 ('have', 2546)]

Wenn es euch auch sinnvoll erscheint, die Html-Escapes wieder rückgängig zu machen, könnt ihr das hier tun.

In [0]:
import html
cleaned_tweet_texts = []
for token in tokens:
  cleaned_tweet_texts.append(html.unescape(token))# TODO
Counter(cleaned_tweet_texts).most_common(20)

[('the', 17252),
 ('to', 11653),
 ('and', 9303),
 ('of', 8246),
 ('a', 7975),
 ('is', 7535),
 ('in', 6777),
 ('for', 5067),
 ('I', 4861),
 ('on', 4842),
 ('be', 4023),
 ('will', 3761),
 ('that', 3217),
 ('are', 3177),
 ('with', 2980),
 ('you', 2963),
 ('&', 2809),
 ('at', 2800),
 ('The', 2759),
 ('have', 2546)]

### 1.6 Verwendete Zeichen
Zum Ende unserer Analysephase schauen wir uns noch an, welche Einzelzeichen in den Tweets vorkommen. Dazu greifen wir wieder auf die konkatenierten (und bereinigten) Tweettexte zurück, die in der Variable ```cleaned_tweet_texts``` gespeichert sind.
Wie viele Zeichen gibt es und wie häufig werden sie verwendet?

In [0]:
cleaned_tweets = ' '.join(cleaned_tweet_texts)
tweet_chars = [char for char in cleaned_tweets]# TODO Tweets als Liste von Zeichen (inklusive Duplikate). "Das hier sind alle Tweets" => ['D', 'a', 's', ' ', 'h', 'i', 'e', ....] (String => Liste)
Counter(tweet_chars).most_common(80)
# TODO: Welche Zeichen kommen vor?
# TODO: Wie oft wird welches Zeichen verwendet?

[(' ', 495487),
 ('e', 251197),
 ('t', 203968),
 ('a', 181386),
 ('o', 180107),
 ('n', 156202),
 ('i', 152156),
 ('r', 141102),
 ('s', 130437),
 ('h', 99586),
 ('l', 95030),
 ('d', 73567),
 ('u', 64669),
 ('c', 63887),
 ('m', 56675),
 ('g', 50691),
 ('p', 50529),
 ('y', 49148),
 ('.', 44782),
 ('w', 42146),
 ('f', 37350),
 ('b', 36134),
 ('/', 25392),
 ('v', 23573),
 ('k', 23007),
 ('T', 22293),
 ('A', 18312),
 (',', 17799),
 ('I', 15825),
 ('C', 15356),
 ('S', 15184),
 ('@', 13742),
 ('!', 13161),
 ('M', 12357),
 ('N', 11580),
 (':', 10938),
 ('R', 10460),
 ('O', 10206),
 ('E', 10181),
 ('W', 9758),
 ('B', 9570),
 ('-', 9134),
 ('D', 8983),
 ('P', 8822),
 ('G', 8544),
 ('H', 8113),
 ('L', 6885),
 ('F', 6682),
 ('0', 6599),
 ('x', 5871),
 ("'", 5395),
 ('1', 5278),
 ('j', 5251),
 ('U', 5235),
 ('J', 5181),
 ('V', 4327),
 ('2', 4030),
 ('Y', 4007),
 ('"', 3794),
 ('#', 3736),
 ('z', 3706),
 ('K', 3686),
 ('’', 3316),
 ('&', 2884),
 ('3', 2617),
 ('6', 2603),
 ('5', 2577),
 ('7', 2370),


### 1.7 Vom Text zur Zeichenliste
Betrachtet die in der vorherigen Teilaufgabe generierte Liste der verwendeten Zeichen. Welche Bereinigungsschritte könnten angemessen sein? Kurze Begründung bitte.

Bevor wir daran gehen, vektorisierte Trainingsdaten für unser Modell zu generieren, wandeln wir unsere Tweets in dieser Teilaufgabe in eine (bereinigte) Liste von Einzelzeichen um und speichern diese in der Variable ```cleaned_tweet_chars```.

```['d', 'i', 'e', 's', ' ', 'i', 's', 't', ' ', 'e', 'i', 'n', ' ', 'b', 'e', 'i', 's', 'p', 'i', 'e', 'l', ',', ' ', 'w', 'i', 'e', ' ', 'd', 'i', 'e', ' ', 'l', 'i', 's', 't', 'e', ' ', 'b', 'e', 'g', 'i', 'n', 'n', 'e', 'n', ' ', 'k', 'ö', 'n', 'n', 't', 'e', '.']```


**Hinweise**: 
* In Python gibt es eine Methode ```str.isprintable()```, die bei der Bereinigung hilfreich sein könnte ...
* Vereinfachung ist legitim. Je mehr Einzelzeichen wir bei der Modellierung berücksichtigen, umso komplexer wird unser Modell und umso höher ist der Trainingsaufwand.

In [0]:
cleaned_tweet_chars = [char.lower() for char in cleaned_tweets if str.isprintable(char)]# TODO Alle Tweets in Einzelzeichen zerlegt und ggf. bereinigt hier rein
#Counter(cleaned_tweet_chars).most_common(57)
cleaned_tweet_chars

['w',
 'i',
 'l',
 'l',
 ' ',
 'b',
 'e',
 ' ',
 'i',
 'n',
 't',
 'e',
 'r',
 'v',
 'i',
 'e',
 'w',
 'e',
 'd',
 ' ',
 'b',
 'y',
 ' ',
 '@',
 'm',
 'a',
 'r',
 'i',
 'a',
 'b',
 'a',
 'r',
 't',
 'i',
 'r',
 'o',
 'm',
 'o',
 ' ',
 'o',
 'n',
 ' ',
 '@',
 'f',
 'o',
 'x',
 'n',
 'e',
 'w',
 's',
 ' ',
 'a',
 't',
 ' ',
 '1',
 '0',
 ':',
 '0',
 '0',
 ' ',
 'a',
 'm',
 '.',
 ' ',
 't',
 'a',
 'l',
 'k',
 'i',
 'n',
 'g',
 ' ',
 'a',
 'b',
 'o',
 'u',
 't',
 ' ',
 't',
 'h',
 'e',
 ' ',
 's',
 'o',
 'u',
 't',
 'h',
 'e',
 'r',
 'n',
 ' ',
 'b',
 'o',
 'r',
 'd',
 'e',
 'r',
 ' ',
 'a',
 'n',
 'd',
 ' ',
 'h',
 'o',
 'w',
 ' ',
 't',
 'h',
 'e',
 ' ',
 'd',
 'e',
 'm',
 's',
 ' ',
 'm',
 'u',
 's',
 't',
 ' ',
 'a',
 'c',
 't',
 ' ',
 'f',
 'a',
 's',
 't',
 ' ',
 't',
 'o',
 ' ',
 'c',
 'h',
 'a',
 'n',
 'g',
 'e',
 ' ',
 'o',
 'u',
 'r',
 ' ',
 'p',
 'a',
 't',
 'h',
 'e',
 't',
 'i',
 'c',
 ' ',
 'i',
 'm',
 'm',
 'i',
 'g',
 'r',
 'a',
 't',
 'i',
 'o',
 'n',
 ' ',
 'l',
 'a',
 'w'

## Aufgabe 2
Nachdem wir uns einen Überblick über den Datensatz verschafft und uns für die Zeichen entschieden haben, die wir bei der Modellierung berücksichtigen wollen, geht es nun darum, die Daten so aufzubereiten, dass wir ein Modell trainieren können.

Das Training soll wie folgt ablaufen: Gegeben n Zeichen, soll das Modell das n+1-te Zeichen vorhersagen. Dazu müssen wir die einzelnen Zeichen in Vektorform bringen. Wir wählen dazu ein One-Hot-Encoding und bilden folglich jedes der Einzelzeichen in ```cleaned_tweet_chars``` auf einen Vektor ab, der genau eine 1 enthält.

(Zu) einfaches Beispiel: ```['a', 'b', 'c'] => [(1,0,0), (0,1,0), (0,0,1)]```

### 2.1 Indizes für das One-Hot-Encoding
Um entscheiden zu können, an welcher Stelle wir die 1 setzen, müssen wir jedem Zeichen einen eindeutigen Index zuweisen.
Umgekehrt wollen wir auch zu jedem Index schnell das zugehörige Zeichen ermitteln können. Wir erstellen daher zwei Dictionaries: ```char2index``` für die Abbildung von Zeichen zu Index und ```index2char``` für die umgekehrte Abbildung von Index zu Zeichen.

In [0]:
import collections
top_chars = Counter(cleaned_tweet_chars).most_common(57)
top_chars[0][0]

#char_set = collections.OrderedDict.fromkeys(item for sublist in cleaned_tweet_chars for item in sublist)# TODO Menge aller verwendeten Zeichen nach Bereinigung
print('Anzahl Zeichen: {}'.format(len(top_chars)))
char2index = {word[0]: index for index, word in enumerate(top_chars)}# TODO Abbildung Zeichen => Index
index2char = {index: word[0] for index, word in enumerate(top_chars)}# TODO Abbildung Index => Zeichen
print(char2index)
print(index2char)


cleaned_tweet_chars = [char for char in cleaned_tweet_chars if char in char2index.keys() ]


Anzahl Zeichen: 57
{' ': 0, 'e': 1, 't': 2, 'a': 3, 'o': 4, 'i': 5, 'n': 6, 'r': 7, 's': 8, 'h': 9, 'l': 10, 'd': 11, 'c': 12, 'u': 13, 'm': 14, 'p': 15, 'g': 16, 'y': 17, 'w': 18, 'b': 19, '.': 20, 'f': 21, 'v': 22, 'k': 23, '/': 24, ',': 25, '@': 26, '!': 27, ':': 28, 'j': 29, '-': 30, 'x': 31, '0': 32, "'": 33, '1': 34, 'z': 35, '2': 36, '"': 37, '#': 38, 'q': 39, '’': 40, '&': 41, '3': 42, '6': 43, '5': 44, '7': 45, '4': 46, '8': 47, '9': 48, '“': 49, '”': 50, '?': 51, ')': 52, '(': 53, '_': 54, '$': 55, '%': 56}
{0: ' ', 1: 'e', 2: 't', 3: 'a', 4: 'o', 5: 'i', 6: 'n', 7: 'r', 8: 's', 9: 'h', 10: 'l', 11: 'd', 12: 'c', 13: 'u', 14: 'm', 15: 'p', 16: 'g', 17: 'y', 18: 'w', 19: 'b', 20: '.', 21: 'f', 22: 'v', 23: 'k', 24: '/', 25: ',', 26: '@', 27: '!', 28: ':', 29: 'j', 30: '-', 31: 'x', 32: '0', 33: "'", 34: '1', 35: 'z', 36: '2', 37: '"', 38: '#', 39: 'q', 40: '’', 41: '&', 42: '3', 43: '6', 44: '5', 45: '7', 46: '4', 47: '8', 48: '9', 49: '“', 50: '”', 51: '?', 52: ')', 53: '(', 

### 2.2 Generiere Trainingsdaten
Wie bereits beschrieben, soll das Modell n Zeichen entgegennehmen und das n+1-te Zeichen vorhersagen. Wir generieren uns Trainingsdaten, indem wir ```sentences``` eine Liste von ```input_length``` Zeichen aus ```cleaned_tweet_chars``` hinzufügen, ```next_chars``` das darauf folgende ```input_length+1```-te Zeichen und dies alle ```step``` Zeichen wiederholen.

In [0]:
input_length = 30
step = 4
sentences = []
next_chars = []


for i in range(0, len(cleaned_tweet_chars) - (input_length + 1), 4):
    sentences.append(cleaned_tweet_chars[i:i + input_length])
    next_chars.append(cleaned_tweet_chars[i + input_length + 1])
print('Anzahl Trainingssätze: {}'.format(len(sentences)))

Anzahl Trainingssätze: 777294


### 2.3 Vektorisierung
Nun können wir daran gehen, die Daten zu vektorisieren. Die Eingabe ```x``` enthält für jeden der Trainingssätze in ```sentences``` one-hot-codierte Vektoren der Zeichen, aus denen der jeweilige Satz besteht. 
Die erwartete Ausgabe ```y``` basiert auf ```next_chars``` und enthält für jeden der Trainingssätze einen einzelnen one-hot-codierten Vektor für das als Fortsetzung des Satzes erwartete Zeichen.

Beispiel: Wenn ```sentences``` an Position ```i``` die Sequenz ```['a', 'b', 'c']``` und ```next_chars``` an derselben Stelle ```'d'``` enthält, dann enthält ```x``` bei entsprechender Kodierung an Position ```i``` ```[[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0]]``` und ```y``` an der entsprechenden Stelle ```[0, 0, 0, 1]```.

In [0]:
import numpy as np

print('Vektorisierung ...')
x = np.zeros((len(sentences), input_length, len(char2index)), dtype=np.bool)
y = np.zeros((len(sentences), len(char2index)), dtype=np.bool)

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i][t][char2index[char]] = 1
    y[i][char2index[next_chars[i]]] = 1
print(y)

Vektorisierung ...
[[False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]
 ...
 [False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]]


### 2.4 Definition des Modells

Für unseren Tweet-Generator werden wir ein recht einfaches Modell trainieren. Wir verwenden wieder die Sequential-API. 

Der erste Layer ist direkt das Herzstück unseres Modells: Der LSTM-Layer. 

Als Anzahl der Units verwenden wir 128.
Welche Dimensionen hat die ```input_shape```? 

In [0]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras import metrics

number_chars = len(char2index)

print('Erstelle Model...')
model = Sequential()
model.add(LSTM(128, input_shape=(input_length, number_chars)))

Using TensorFlow backend.


Erstelle Model...





Als Ausgabelayer fügen wir einen Dense-Layer hinzu. Wie müssen wir die Anzahl der Hidden-Units wählen? Wieso ist ```softmax``` eine geeignete Aktivierungsfunktion?

In [0]:
model.add(Dense(number_chars, activation = 'softmax'))

Als Optimizer wählen wir RMSprop mit einer Learning-Rate von 0.005 und als Loss-Funktion ```categorical_crossentropy```.

Wieso können wir nicht wie im letzten Labor ```binary_crossentropy``` verwenden?

In [0]:
optimizer = RMSprop(lr=0.005)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

print(model.summary())



Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               95232     
_________________________________________________________________
dense_1 (Dense)              (None, 57)                7353      
Total params: 102,585
Trainable params: 102,585
Non-trainable params: 0
_________________________________________________________________
None


### 2.5 Training des Models

Nach all den Vorarbeiten können wir nun daran gehen, unser Model zu trainieren.
Um das Model bei Bedarf nicht neu definieren zu müssen, speichern wir uns dessen Struktur als JSON ab.

In [0]:
model_structure = model.to_json()
with open("/data/My Drive/Colab Notebooks/data/text_generation_model.json", "w") as json_file:
    json_file.write(model_structure)

Um unsere Trainingsfortschritte nicht zu verlieren, definieren wir uns eine Checkpoint-Funktion, die als Callback aufgerufen wird und den aktuellen Modelzustand abspeichert, solange das Modell besser ist, als das bisher gespeicherte Model. Gespeichert werden soll das komplette Model, nicht nur die Gewichte.

In [0]:
from keras.callbacks import ModelCheckpoint
model_checkpoint = ModelCheckpoint('/data/My Drive/Colab Notebooks/data/text_generation.hd5', monitor='acc', save_best_only=True)# TODO nur bestes Model speichern, komplettes Model speichern

Wir können unser Model nun trainieren. Um in ansehbarer Zeit Ergebnisse zu sehen, führen wir unser Training über 20 Epochen mit eine Batch-Size von 100 durch.

Um die Entwicklung des Models später auswerten zu können, speichern wir uns die History.

In [0]:
import pickle

epochs = 20
batch_size = 100


model_history = model.fit(x, y, epochs=epochs, batch_size=batch_size, callbacks=[model_checkpoint]) 
# TODO Model fit mit Trainingsdaten, angegebenen Parametern und checkpoint

with open("/data/My Drive/Colab Notebooks/data/text_generation_history", 'wb') as hist_file:
    pickle.dump(model_history.history, hist_file)
print('History gespeichert.')

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/20





777294/777294 [==============================] - 465s 598us/step - loss: 2.4475 - acc: 0.3290
Epoch 2/20
777294/777294 [==============================] - 491s 632us/step - loss: 2.2183 - acc: 0.3902
Epoch 3/20
777294/777294 [==============================] - 487s 627us/step - loss: 2.1595 - acc: 0.4051
Epoch 4/20
777294/777294 [==============================] - 494s 635us/step - loss: 2.1294 - acc: 0.4125
Epoch 5/20
777294/777294 [==============================] - 490s 630us/step - loss: 2.1103 - acc: 0.4173
Epoch 6/20
777294/777294 [==============================] - 491s 632us/step - loss: 2.0982 - acc: 0.4203
Epoch 7/20
777294/777294 [==============================] - 492s 633us/step - loss: 2.0892 - acc: 0.4234
Epoch 8/20
777294/777294 [==============================] - 487s 627us/step - loss: 2.0796 - acc: 0.4260
Epoch 9/20
777294/777294 [==============================] 

## Aufgabe 3
Zum Abschluss wollen wir noch etwas Spaß mit unserem Modell haben. Aus diesem Grund laden wir uns das gespeicherte (bisher) beste Model und verwenden es, um basierend auf einem "Seed" neue Tweets zu generieren.

In [0]:
from keras.models import load_model
loaded_model = load_model('text_generation.hd5')

### Eine kleine Hilfsfunktion
Der Ausgabelayer unseres Models beschreibt eine Wahrscheinlichkeitsverteilung über alle möglichen Ausgabezeichen. Was wir tun wollen, ist anhand dieser Verteilung eine repräsentative Stichprobe zu ziehen. Ähnlich wie bei Markov-Ketten wählen wir als nächstes Zeichen nicht zwangsläufig das, mit der höchsten Auftrittswahrscheinlichkeit, denn wir wollen uns ja eine gewissen künstlerisch-kreative Freiheit erhalten, aber wir orientieren uns bei der Auswahl an der Auftrittswahrscheinlichkeit der potentiellen Folgetokens im gegebenen Kontext.

Die Hilfsmethode, die wir dazu verwenden, ist aus dem Keras-Tutorial zu LSTMs übernommen. Der Parameter ```temperature``` schärft die ursprüngliche Wahrscheinlichkeitsverteilung oder schwächt sie ab. Bei ```temperature > 1``` ist die Ausgabe diverser, allerdings potentiell auch konfuser, bei ```temperature < 1 ``` bleiben wir näher an den Originaltweets.

```multinominal(num_samples, probabilities_list, size``` zieht ```size```-mal ```num_samples``` Beispiele aus einer Verteilung, deren Eigenschaften durch ```probabilities_list``` beschrieben wird. In unserem Fall wollen wir einmal ziehen und zwar genau ein Beispiel. Ausgabe ist demnach eine Liste, die einen einzigen Vektor enthält, der genauso lang ist, wie die Wahrscheinlichkeitsverteilung, die wir in die Funktion hineingeben, und der eine einzige 1 enthält. Für den Index dieser 1 interessieren wir uns, weil wir das entsprechend one-hot-codierte Zeichen als nächstes ausgeben wollen.

In [0]:
import random

def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds.clip(min=0.0001)) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

### 3.1 Tweets erzeugen
Um neue Tweets zu erzeugen, brauchen wir einen Seed, mit dem unser Model arbeiten kann. Wir machen es uns einfach und wählen einen zufälligen Startindex, ab dem ```input_length``` Zeichen aus unserer langen Tweet-Liste ```cleaned_tweet_chars``` herausgenommen werden.

Diese vektorisieren wir dann und füttern unser Model mit dem so entstandenen Vektor, um das nächste Zeichen vorherzusagen, das dann wiederum Teil des Seeds für die nächste Vorhersage ist.


In [0]:
import sys

seed_start_index = # TODO Zufallszahl für Startindex innerhalb von cleaned_tweet_chars wählen

for diversity in [0.2, 0.5, 0.8, 1.0, 1.2]:
    print()
    print('----- Diversität:', diversity)

    generated = ''
    sentence = # Subliste der passenden Länge aus cleaned_tweet_charts extrahieren
    generated += ''.join(sentence)
    print('----- Erzeuge Tweet aus Seed: "' + ''.join(sentence) + '"\n')
    sys.stdout.write(generated)

    for i in range(250):
        x = np.zeros((1, #TODO 2. Dimension?, len(char_set)))
        for t, char in enumerate(sentence):
            x[0, t, # TODO Wo muss die 1 hin?] = 1.

        preds = loaded_model.predict(x, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = # TODO Welchem Zeichen entspricht der Index?

        generated += next_char
        sentence = # TODO sentence bleibt eine Liste von Zeichen, aber das erste fällt weg und next_char kommt am Ende neu hinzu

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()

### 3.2 Entwicklung von Loss und Accuracy
Um die Aufgabe abzuschließen, wollen wir noch einen Blick auf die Entwicklung von Loss und Accuracy über die Trainingsepochen werfen. Wir haben die "Geschichte" unseres Models in der Datei ```text_generation_history``` abgespeichert.

Nutzt ```pyplot```, um die Entwicklung von Accuracy und Loss grafisch darzustellen.

In [0]:
import matplotlib.pyplot as plt

with open("text_generation_history", 'rb') as hist_file:
    history = pickle.load(hist_file)
    
plt.plot(# Entwicklung der Accuracy)
plt.title('Accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.show()

# TODO Loss plotten
